<a href="https://www.kaggle.com/code/khelifayacine/playground-series-saison-4-pisode-10?scriptVersionId=200148783" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Exploration des données :

In [ ]:
# Importation des packages

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# Package modélisation 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV




In [ ]:
# Importation des données 
df_train = pd.read_csv("/kaggle/input/playground-series-s4e10/train.csv")
df_test = pd.read_csv ("/kaggle/input/playground-series-s4e10/test.csv")
df_train.head(10)

Description des colonnes :
- id : Identifiant unique pour chaque observation.
- person_age : Âge du demandeur de prêt.
- person_income : Revenu annuel du demandeur.
- person_home_ownership : Statut de propriété du logement (par exemple : RENT, OWN, MORTGAGE).
- person_emp_length : Nombre d'années d'emploi du demandeur.
- loan_intent : Raison du prêt (par exemple : EDUCATION, MEDICAL, VENTURE).
- loan_grade : Classement du prêt attribué (par exemple : A, B, C).
- loan_amnt : Montant demandé pour le prêt.
- loan_int_rate : Taux d'intérêt du prêt.
- loan_percent_income : Proportion du revenu du demandeur par rapport au montant du prêt.
- cb_person_default_on_file : Indicateur de défaut de paiement dans l'historique de crédit du demandeur (Y/N).
- cb_person_cred_hist_length : Durée de l'historique de crédit du demandeur (en années).
- loan_status : Statut du prêt (0 pour refusé, 1 pour approuvé).

In [ ]:
df_test.head(10)

In [ ]:
df_train.info()

In [ ]:
#Analyse de valeurs manquantes
df_train.isna().sum()#aucune valeure manquante !

In [ ]:
#Analyse des statistiques descriptives :
df_train.describe()

In [ ]:
df_train.duplicated().sum()

## DataViz

1. Visualisation de la distribution de la cible (loan_status) :

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(data=df_train, x='loan_status')
plt.title('Distribution de la variable cible (loan_status)')
plt.xlabel('Loan Status (0: Non-Approved, 1: Approved)')
plt.ylabel('Count')
plt.show()

2. Vérification des corrélations entre les variables numériques :

In [ ]:
# Sélectionner les colonnes numériques uniquement
numerical_cols = df_train.select_dtypes(include=['int64', 'float64'])

# Calculer la matrice de corrélation
corr_matrix = numerical_cols.corr()

# Afficher la matrice de corrélation
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matrice de corrélation entre les variables numériques')
plt.show()



Étape 2 : Visualisation des Relations

3) Visualisation des relations entre les caractéristiques et la cible :

Age et Loan Status :

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(data=df_train, x='loan_status', y='person_age')
plt.title('Âge des demandeurs selon le statut du prêt')
plt.xlabel('Loan Status')
plt.ylabel('Age')
plt.xticks([0, 1], ['Non-Approved', 'Approved'])
plt.show()


Revenu et Loan Status :

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(data=df_train, x='loan_status', y='person_income')
plt.title('Revenu des demandeurs selon le statut du prêt')
plt.xlabel('Loan Status')
plt.ylabel('Income')
plt.xticks([0, 1], ['Non-Approved', 'Approved'])
plt.show()


4) Visualisation des catégories :

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=df_train, x='person_home_ownership', hue='loan_status')
plt.title('Propriétés de logement selon le statut du prêt')
plt.xlabel('Home Ownership')
plt.ylabel('Count')
plt.legend(title='Loan Status', loc='upper right', labels=['Non-Approved', 'Approved'])
plt.show()


In [ ]:

sns.boxplot(df_train['person_income'])
plt.title('Distribution du revenu')
plt.show()


Visualisation des distributions avec KDE (Estimation de Densité Noyau) :

In [ ]:
sns.kdeplot(df_train, x='person_income',hue='loan_status')
plt.title('Distribution du revenu par statut du prêt')
plt.show()

Statistiques sur les outliers : 
- Calculer le nombre d'observations qui sont considérées comme des outliers. Cela peut être fait en utilisant l'écart interquartile (IQR) pour définir des seuils.

In [ ]:
Q1 = df_train['person_income'].quantile(0.25)
Q3 = df_train['person_income'].quantile(0.75)
IQR = Q3 - Q1

outliers = df_train[(df_train['person_income'] < (Q1 - 1.5 * IQR)) | (df_train['person_income'] > (Q3 + 1.5 * IQR))]
print(f"Nombre d'outliers dans person_income : {len(outliers)}")


In [ ]:
sns.boxplot(df_train[df_train['person_income'] < (Q3 + 1.5 * IQR)]['person_income'])
plt.title('Distribution du revenu sans outliers')
plt.show()


Distribution log-transformée :
- Si les valeurs extrêmes sont légitimes et qu'il est nécessaire de travailler avec ces données, appliquer une transformation logarithmique peut aider à normaliser la distribution.

In [ ]:
df_train['log_person_income'] = np.log1p(df_train['person_income'])
sns.histplot(df_train['log_person_income'], kde=True)
plt.title('Distribution du revenu (log-transformée)')
plt.show()


Résumé des Étapes d'EDA :
- Visualisation de la distribution de la variable cible pour voir la balance des classes.
- Analyse descriptive pour comprendre les caractéristiques des données.
- Corrélations pour identifier les relations entre les variables.
- Visualisation des relations entre les caractéristiques et la cible pour mieux comprendre l'impact de chaque variable.

# Preprocessing de données 

Encodage des variables df_train

In [ ]:
label_encoder = LabelEncoder()
df_train['person_home_ownership'] = label_encoder.fit_transform(df_train['person_home_ownership'])
df_train['loan_intent'] = label_encoder.fit_transform(df_train['loan_intent'])
df_train['loan_grade'] = label_encoder.fit_transform(df_train['loan_grade'])
df_train['cb_person_default_on_file'] = label_encoder.fit_transform(df_train['cb_person_default_on_file'])

In [ ]:
df_train.info()



Encodage du dataset df_test

In [ ]:
df_test['person_home_ownership'] = label_encoder.fit_transform(df_test['person_home_ownership'])
df_test['loan_intent'] = label_encoder.fit_transform(df_test['loan_intent'])
df_test['loan_grade'] = label_encoder.fit_transform(df_test['loan_grade'])
df_test['cb_person_default_on_file'] = label_encoder.fit_transform(df_test['cb_person_default_on_file'])


In [ ]:
df_test.info()

In [ ]:
# Vérifier si les colonnes de df_train (sans la cible) et df_test correspondent
train_columns = df_train.drop(columns=['loan_status']).columns
test_columns = df_test.columns

# Vérification des différences
if set(train_columns) != set(test_columns):
    print("Les colonnes entre df_train et df_test ne correspondent pas.")
    print("Colonnes manquantes dans df_test :", set(train_columns) - set(test_columns))
    print("Colonnes supplémentaires dans df_test :", set(test_columns) - set(train_columns))
else:
    print("Les colonnes correspondent.")


5. Séparation des données en train/test (Validation) :

Avant d'entraîner ton modèle, il est nécessaire de séparer notre dataset en données d'entraînement et de test pour éviter le surapprentissage et tester la généralisation du modèle.

In [ ]:
from sklearn.model_selection import train_test_split

# Séparation des données (80% train, 20% test)
X = df_train.drop('loan_status', axis=1)
y = df_train['loan_status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Modélisation 

1. Choix du modèle :

Plusieurs modèles peuvent être testés, par exemple :
- Régression Logistique
- Arbres de décision
- Random Forest
- Gradient Boosting
- XGBoost
- LightGBM

In [ ]:
from sklearn.model_selection import cross_val_score

# Logistic Regression
model_log_reg = LogisticRegression(max_iter=1000)
model_log_reg.fit(X, y)
log_reg_acc = cross_val_score(model_log_reg, X, y, cv=5, scoring='accuracy').mean()
log_reg_roc_auc = cross_val_score(model_log_reg, X, y, cv=5, scoring='roc_auc').mean()

print(f"Logistic Regression - Accuracy: {log_reg_acc:.4f}, ROC AUC (CV 5 folds): {log_reg_roc_auc:.4f}")

In [ ]:
# Random Forest
model_rf = RandomForestClassifier()
model_rf.fit(X, y)
rf_acc = cross_val_score(model_rf, X, y, cv=5, scoring='accuracy').mean()
rf_roc_auc = cross_val_score(model_rf, X, y, cv=5, scoring='roc_auc').mean()

print(f"Random Forest - Accuracy: {rf_acc:.4f}, ROC AUC (CV 5 folds): {rf_roc_auc:.4f}")

In [ ]:
# KNN
model_knn = KNeighborsClassifier()
model_knn.fit(X, y)
knn_acc = cross_val_score(model_knn, X, y, cv=5, scoring='accuracy').mean()
knn_roc_auc = cross_val_score(model_knn, X, y, cv=5, scoring='roc_auc').mean()

print(f"KNN - Accuracy: {knn_acc:.4f}, ROC AUC (CV 5 folds): {knn_roc_auc:.4f}")

In [ ]:
# Decision Tree
model_dt = DecisionTreeClassifier()
model_dt.fit(X, y)
dt_acc = cross_val_score(model_dt, X, y, cv=5, scoring='accuracy').mean()
dt_roc_auc = cross_val_score(model_dt, X, y, cv=5, scoring='roc_auc').mean()

print(f"Decision Tree - Accuracy: {dt_acc:.4f}, ROC AUC (CV 5 folds): {dt_roc_auc:.4f}")

In [ ]:
# XGBoost
model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model_xgb.fit(X, y)
xgb_acc = cross_val_score(model_xgb, X, y, cv=5, scoring='accuracy').mean()
xgb_roc_auc = cross_val_score(model_xgb, X, y, cv=5, scoring='roc_auc').mean()

print(f"XGBoost - Accuracy: {xgb_acc:.4f}, ROC AUC (CV 5 folds): {xgb_roc_auc:.4f}")

## Optimiser Tous les Modèles

In [ ]:
# Afficher l'importance des variables pour la régression logistique
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X, y)

# Importance des variables à partir des coefficients
importances = np.abs(logistic_model.coef_[0])
indices = np.argsort(importances)[::-1]

# Tracer l'importance des variables
plt.figure(figsize=(10, 6))
plt.title("Importance des Variables - Logistic Regression")
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), X.columns[indices], rotation=90)
plt.xlim([-1, X.shape[1]])
plt.ylabel("Coefficient")
plt.xlabel("Variables")
plt.show()

In [ ]:
# Afficher l'importance des variables pour Random Forest
random_forest_model = RandomForestClassifier()
random_forest_model.fit(X, y)

# Importance des variables
importances = random_forest_model.feature_importances_
indices = np.argsort(importances)[::-1]

# Tracer l'importance des variables
plt.figure(figsize=(10, 6))
plt.title("Importance des Variables - Random Forest")
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), X.columns[indices], rotation=90)
plt.xlim([-1, X.shape[1]])
plt.ylabel("Importance")
plt.xlabel("Variables")
plt.show()

In [ ]:
# Afficher l'importance des variables pour Decision Tree
decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(X, y)

# Importance des variables
importances = decision_tree_model.feature_importances_
indices = np.argsort(importances)[::-1]

# Tracer l'importance des variables
plt.figure(figsize=(10, 6))
plt.title("Importance des Variables - Decision Tree")
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), X.columns[indices], rotation=90)
plt.xlim([-1, X.shape[1]])
plt.ylabel("Importance")
plt.xlabel("Variables")
plt.show()

In [ ]:
# Afficher l'importance des variables pour XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X, y)

# Importance des variables
importances = xgb_model.feature_importances_
indices = np.argsort(importances)[::-1]

# Tracer l'importance des variables
plt.figure(figsize=(10, 6))
plt.title("Importance des Variables - XGBoost")
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), X.columns[indices], rotation=90)
plt.xlim([-1, X.shape[1]])
plt.ylabel("Importance")
plt.xlabel("Variables")
plt.show()

### Conclusion sur le Choix du Modèle
Choisir Random Forest pour les prédictions sur df_test est judicieux, notamment parce que ce modèle excelle dans la gestion de datasets comportant de nombreuses variables, tout en offrant des performances solides grâce à son approche basée sur des arbres décisionnels multiples. De plus, il est capable de capter des relations complexes et des interactions entre les variables sans nécessiter de transformation extensive des données.


# Prediction sur le Test_csv

1. Prédiction RandomForest

In [ ]:
#  RandomForestClassifier 
best_model = RandomForestClassifier()

# Entraînement du modèle sur df_train
best_model.fit(X_train, y_train)

# Faire des prédictions sur df_test
y_test_pred = best_model.predict(df_test)

# Si le modèle supporte les probabilités, obtenir les probabilités:
y_test_pred_proba = best_model.predict_proba(df_test)[:, 1]

# Créer un fichier de soumission
submission = pd.DataFrame({
    'id': df_test['id'],  
    'loan_status': y_test_pred  # Ou utiliser y_test_pred_proba si tu veux soumettre des probabilités
})

# Sauvegarder le fichier de soumission
chemin_fichier = '/kaggle/working/submission.csv'
submission.to_csv(chemin_fichier, index=False)
print(f"Fichier de soumission créé avec succès dans {chemin_fichier}")


In [ ]:
# Afficher les 20 premières lignes des prédictions
result = pd.DataFrame({
    'id': df_test['id'],
    'loan_status_pred': y_test_pred
})
print(result.head(20))

In [ ]:
# Prédictions sur df_test
y_test_pred = best_model.predict(df_test)

# Affichage des résultats
result_counts = pd.Series(y_test_pred).value_counts()

# Créer un graphique à barres
plt.figure(figsize=(8, 5))
sns.barplot(x=result_counts.index, y=result_counts.values, palette='viridis')

# Ajouter les labels et le titre
plt.title('Prédictions de Random Forest sur df_test')
plt.xlabel('Loan Status')
plt.ylabel('Count')
plt.xticks(ticks=result_counts.index, labels=result_counts.index, rotation=0)

# Ajouter le pourcentage au-dessus de chaque barre
total_count = result_counts.sum()
for index, value in enumerate(result_counts):
    percentage = (value / total_count) * 100
    plt.text(index, value, f'{percentage:.1f}%', ha='center', va='bottom')

plt.show()

2. Prédiction XGBOOST

In [ ]:
from xgboost import XGBClassifier

# Entraîner XGBoost sur df_train
best_model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Entraînement du modèle sur les données d'entraînement
best_model_xgb.fit(X_train, y_train)

# Faire des prédictions sur df_test
y_test_pred_xgb = best_model_xgb.predict(df_test)

# Si tu veux aussi les probabilités :
y_test_pred_proba_xgb = best_model_xgb.predict_proba(df_test)[:, 1]

# Créer un fichier de soumission pour XGBoost
submission_xgb = pd.DataFrame({
    'id': df_test['id'],  
    'loan_status': y_test_pred_xgb
})

# Sauvegarder le fichier de soumission en CSV
submission_xgb.to_csv('submission_XGBoost.csv', index=False)

print("Fichier de soumission XGBoost créé avec succès.")

In [ ]:
# Afficher les 20 premières lignes des prédictions XGBoost
result_xgb = pd.DataFrame({
    'id': df_test['id'],
    'loan_status_pred': y_test_pred_xgb
})
print(result_xgb.head(20))

In [ ]:
# Créer un DataFrame pour résumer les prédictions XGBoost
predictions_summary_xgb = result_xgb['loan_status_pred'].value_counts()

# Calculer les pourcentages pour chaque classe
percentages = predictions_summary_xgb / predictions_summary_xgb.sum() * 100

# Afficher les valeurs des prédictions
print(predictions_summary_xgb)

# Créer un graphique à barres pour XGBoost
ax = predictions_summary_xgb.plot(kind='bar', color=['lightgreen', 'orange'])

# Ajouter les pourcentages ou les valeurs au-dessus des barres
for i in ax.containers:
    ax.bar_label(i, labels=[f'{val:.1f}%' for val in percentages], label_type='edge')

# Ajouter les titres et étiquettes
plt.title('Répartition des prédictions avec XGBoost - Prêt accepté vs rejeté')
plt.xlabel('Statut du prêt (0 = rejeté, 1 = accepté)')
plt.ylabel('Nombre de prédictions')
plt.xticks(rotation=0)

# Afficher le graphique
plt.show()